# Dashboard

This pipeline processes data into a form to drive the dashboard.

The parameters are injected using papermill.

In [1]:
target: str = ''
start: str = '2023-01-01'
end: str = '2025-06-10'
fixed_date: str = 'true'

## Setup

Import some libraries

In [2]:
import json
from datetime import date
from ast import literal_eval

import petl as etl
from utils.paths import PUBLISHED, SITE
from utils.themes.programme import ProgrammeSlice

Set the target directory, and ensure it exists

In [3]:
TARGET = SITE / 'insights/dashboard' / target / '_data'
TARGET.mkdir(exist_ok=True, parents=True)

Calculate the date range. If the `fixed_date` parameter is `false`, make today the latest date.

In [4]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end),
    date.today(),
)

Write the timestamp file

In [5]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end', 'updated'], (str(x) for x in date_range))), f)

Load programme data

In [6]:
programme_data = ProgrammeSlice(range=date_range[0:2])

## Events & projects data

In [7]:
events = (
    programme_data.project_breakdown
    .aggregate(['project_name', 'project_id', 'evaluation_category'], sum, 'events', field='events')
)

In [8]:
project_summary = {
    'total': events.distinct('project_name').nrows(),
    'by_category': dict(events.aggregate('evaluation_category', len).records()),
    'names': list(events.cut('project_name').distinct().values('project_name')),
}

In [9]:
events_summary = {
    'total': sum(events.values('events')),
    'by_category': dict(events.aggregate('evaluation_category', sum, 'events').records()),
    'by_project': {
        r[0]: {'category': r[2], 'count': r[3]}
        for r
        in events.records()
    },
}

In [10]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        events_summary,
        f,
        indent=2
    )

with open(TARGET / 'projects.json', 'w') as f:
    json.dump(
        project_summary,
        f,
        indent=2
    )

## Audiences

In [11]:
ticketed_events = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/event-instances.csv')
    .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
    .selectrangeopen('start', date_range[0], date_range[1])
    .selectcontains('eventType', 'MainProgrammeEvent')
)

In [12]:
tickets_sold = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(ticketed_events.cut('instance_id', 'event_name'))
    .cut('event_name', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
)

In [13]:
tickets_scanned = dict(
    etl
    .fromcsv(PUBLISHED / 'manual/spektrix-sales-and-scans.csv')
    .convert('Event Date', etl.dateparser('%Y-%m-%d'))
    .selectrangeopen('Event Date', date_range[0], date_range[1])
    .convertnumbers()
    .aggregate(None, {
        'sold': ('Sold', sum),
        'scanned': ('Scanned', sum)
    })
    .melt(variables=['sold', 'scanned'])
    .records()
)

In [14]:
ticketed_audience = {
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
        'by_project': dict(tickets_sold.aggregate('event_name', sum, 'count_of_tickets', field='count_of_tickets').records()),
        'by_geo': dict(tickets_sold.aggregate('geography_code', sum, 'count_of_tickets', field='count_of_tickets').records()),
    },
    'scanned': {
        'total': tickets_scanned
    }
}

In [15]:
audience_data = (
    programme_data.project_breakdown
    .aggregate(['project_name', 'project_id', 'evaluation_category'], sum, 'audience', field='audience')
)


In [16]:
audience = {
    'total': sum(audience_data.values('audience')),
    'by_category': dict(
        audience_data
        .aggregate(['evaluation_category'], sum, 'audience')
        .records()
    ),
    'by_project': dict(
        audience_data
        .aggregate(['project_name'], sum, 'audience')
        .records()
    ),
}

In [17]:
with open(TARGET / 'audience.json', 'w') as f:
    json.dump(
        audience | {
            'ticketed': ticketed_audience,
            'unticketed': None,
        },
        f,
        indent=2
    )

## Participants

### Programme

In [18]:
programme_participants = (
    programme_data.project_breakdown
    .melt(variables=[
        'schedule_participants_community',
        'manual_participants_community',
        # 'manual_participants_schools'
    ])
    .convert('variable', lambda f: f.replace('manual_','').replace('schedule_',''))
    .selectnotnone('value')
    .aggregate(['project_name', 'project_id', 'evaluation_category', 'variable'], sum, 'value')
    .recast(samplesize=1_000_000)
)

### Cultural learning

In [19]:
import petl as etl
from datetime import date
import ast


cultural_learning_engagements = (
    etl
    .fromcsv(PUBLISHED / 'cultural-learning/engagements.csv' )
    .convert(['date', 'month'], date.fromisoformat)
    .selectnotnone('date')
    .selectrangeopen('date', *date_range[:2])
    .convertnumbers()
    .convert('organisation_ids', ast.literal_eval)
)

def distinct(items):
    return len({i for s in items for i in s})

cultural_learning_summary = (
    cultural_learning_engagements
    .aggregate(None, {
        'participants': ('participants', sum),
        'schools_engaged': ('organisation_ids', distinct)
    })
    .dicts()[0]
)



### Volunteers

In [20]:
volunteer_signups = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/checkpoint-updates.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selectle('date', date_range[1])
    .convertnumbers(strict=False)
)

In [21]:
# volunteer_shifts = (
#     etl
#     .fromcsv(PUBLISHED / 'volunteers/shifts.csv')
#     .convert('date', etl.dateparser('%Y-%m-%d'))
#     .selecteq('type', 'BD25 Event')
#     .selectrangeopen('date', date_range[0], date_range[1])
#     .convertnumbers(strict=False)
# )

In [22]:
volunteer_summary = {
    'total': sum(
        volunteer_signups
        .selectin('checkpoint',  ["1. Monitoring & Evaluation", "2. Sign Up to Induction", "3. Fully Inducted Volunteers"])
        .values('count')
    ),
    'people': {
        'checkpoints': dict(volunteer_signups.aggregate('checkpoint', sum, 'count').records()),
    },
    # 'events': {
    #     'count': volunteer_shifts.cut('rosterfy_event_name').distinct().nrows(),
    #     'names': list(volunteer_shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    # },
    # 'shifts': {
    #     'count': volunteer_shifts.nrows(),
    #     'attended': sum(volunteer_shifts.values('attended')),
    #     'hours': sum(volunteer_shifts.values('hours')),
    # },
}

### Other figures

In [23]:
other_figures = etl.fromcsv(PUBLISHED / 'manual/manual-other-figures.csv').selecteq('dashboard key', target).convertnumbers()

In [24]:
dict(other_figures.selecteq('variable', 'training_and_skills_development_participants').cut('source', 'value').rename('source', 'project_name').aggregate('project_name', sum, 'value', field='count').records())

{'Bradford Producing Hub': 833,
 'Creative Health': 278,
 'Digital Creatives scheme': 1632}

In [25]:
with open(TARGET / 'otherFigures.json', 'w') as f:
    json.dump(
        other_figures.aggregate('variable', sum, 'value').transpose().cutout('variable').dicts()[0],
        f,
        indent=2
    )

### Summary

In [26]:
participants_by_type = (
    programme_participants
    .melt(key=['project_name', 'project_id', 'evaluation_category'])
    .selectnotnone('value').aggregate(['variable'], sum, 'value')
    .transpose().cutout('variable').dicts()[0]
)

In [ ]:
def summarise_by_type(table):
    return {
        'count': sum(table.values('count')),
        'by_project': dict(table.selectgt('count', 0).records()),
    }

by_type = {
    'community_participants': summarise_by_type(programme_participants
        .cutout('project_id', 'evaluation_category')
        .rename('participants_community', 'count')
    ),
    'training_and_skills_development_participants': summarise_by_type(
        other_figures
        .selecteq('variable', 'training_and_skills_development_participants')
        .cut('source', 'value')
        .rename('source', 'project_name')
        .aggregate('project_name', sum, 'value', field='count')   
    ),
    'cultural_learning_participants': summarise_by_type(
        cultural_learning_engagements.aggregate('project_name', sum, 'participants', field='count')
    )
}

In [28]:
by_type

{'community_participants': {'count': 48853,
  'by_project': {'A Screen Near You - Cinema for All (grassroots development and screening)': 0,
   'A Screen Near You - The Incredible Moving Cinema': 0,
   'ARCHIVE: How Soon Is Now (grant dependent)': 0,
   'Akram Khan and Dance United - Memories of the Future': 0,
   'An Evening with Resi/Dance @ The Beacon (Dance United)': 0,
   'An Evening with: Jacqui Dankworth': 0,
   'Aswad Arts Presents StreetSounds "LIVE" Bhangra Niche Set "LIVE"': 0,
   'BBC Extraordinary Portraits': 2400,
   'BBC Introducing @ The Beacon': 9,
   'BBC Introducing at The Underground': 0,
   'BD: Walls': 254,
   'Basel Zaraa - Moomins': 0,
   'Bassline Symphony': 245,
   'Beacon': 0,
   'Beacon Church Community Cuppa': 0,
   'Beacon Feature Curtain': 0,
   'Big Brass Blowout': 33,
   'Big Screen - Cannon Mills': 0,
   'Bloom Vicky Amudeme': 0,
   'Bradford Belles': 0,
   'Bradford Literature Festival - New Commissions': 0,
   'Bradford Made MASTER': 0,
   'Bradford 

In [29]:
participant_summary = {
    'total': sum(programme_participants.values('participants_community'))
        + cultural_learning_summary['participants']
        + volunteer_summary['total']
        + sum(other_figures.selectcontains('variable', '_participants').values('value')),

    'by_type': by_type,

    'cultural_learning': cultural_learning_summary,
    'volunteers': volunteer_summary,
}

In [30]:
with open(TARGET / 'participants.json', 'w') as f:
    json.dump(
        participant_summary,
        f,
        indent=2
    )

## Other figures